#### 【 지도학습 - 회귀 】
- 회귀(Regression) : 수치값을 예측하는 학습
    * 예 : 거리, 시간, 무게, 집가격 등등 ...
- 주제 : 생선 길이에 따른 무게를 예측하는 모델
- 방법 : 최근접이웃 알고리즘

[1] 모듈 로딩 및 데이터 준비 <hr>

In [ ]:
## ================================================
## [1-1] 모듈 로딩
## ================================================
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import koreanize_matplotlib

from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsRegressor
